In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,col
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName("saleComulativeSum").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 15:02:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Sample Data: (employee, sale_date, amount)
sales_data = [
    ("E001", "2024-03-01", 500),
    ("E001", "2024-03-02", 300),
    ("E001", "2024-03-03", 700),
    ("E002", "2024-03-01", 400),
    ("E002", "2024-03-02", 600),
    ("E002", "2024-03-03", 200),
    ("E001", "2024-03-04", 600),
    ("E002", "2024-03-04", 500),
    ("E003", "2024-03-01", 1000),
    ("E003", "2024-03-02", 300),
]

# Creating DataFrame
columns = ["employee", "sale_date", "amount"]

In [4]:
df_sale_comu = spark.createDataFrame(sales_data,columns)

In [ ]:
#select employee,sale_data,amount , sum (amount) over(partition by employee orber by sale_date) as sale_coumn_amount
#from sale_comul_table

In [5]:
window_spec = Window.partitionBy("employee").orderBy(col("sale_date"))

In [7]:
df_sale = df_sale_comu.withColumn("sale_comulative",sum("amount").over(window_spec))

In [8]:
df_sale.show()

+--------+----------+------+---------------+
|employee| sale_date|amount|sale_comulative|
+--------+----------+------+---------------+
|    E001|2024-03-01|   500|            500|
|    E001|2024-03-02|   300|            800|
|    E001|2024-03-03|   700|           1500|
|    E001|2024-03-04|   600|           2100|
|    E002|2024-03-01|   400|            400|
|    E002|2024-03-02|   600|           1000|
|    E002|2024-03-03|   200|           1200|
|    E002|2024-03-04|   500|           1700|
|    E003|2024-03-01|  1000|           1000|
|    E003|2024-03-02|   300|           1300|
+--------+----------+------+---------------+

